In [2]:
import sys
sys.path.append("..")

In [23]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from re import sub
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from nltk import ngrams

from src.utils.clean_comments import CommentCleaner
from src.utils.guess_gender import GenderGuesser

import gspread as gs
import os

In [4]:
file_dir = '../data/comments/'
df_list = []
for fn in [tf for tf in os.listdir(file_dir) if ('.csv' in tf) & (~tf.endswith('_.csv'))]:
    df = pd.read_csv(file_dir+fn)
    df['date_aired'] = pd.to_datetime(fn.split('_')[-1].split('.')[0]).strftime('%Y-%m-%d')
    df_list.append(df[['date_aired','sender_name','text']])

In [5]:
all_df = pd.concat(df_list).reset_index().rename(columns={'index':'comment_id'})
all_df['comment_id'] = all_df['comment_id']+61
all_df['comment_id']= all_df.apply(lambda x: \
                                   x['date_aired'].replace('-','')+str(x['comment_id']).rjust(4, '0'), axis=1)
all_df

,comment_id,date_aired,sender_name,text
0,202207070061,2022-07-07,John Godfrey,Palala lamang po sa mga ofws sa ksa wag mameke...
1,202207070062,2022-07-07,Rowena Ascrate,"Hello,watching po from uae"
2,202207070063,2022-07-07,Zhai Vhie Gimoro Francisco,Help nyo po aq sir arnel
3,202207070064,2022-07-07,Cbbc Fort Bonifacio SharjahOutreach,Good evening po Sirs and Mam
4,202207070065,2022-07-07,Agripina Esmajer,Hi Sir Arnel good evening Po!watching from San...
...,...,...,...,...
26214,202207211605,2022-07-21,Sarupac Jessa,Magandang araw po saan po pwede lumapit kung m...
26215,202207211606,2022-07-21,Sarupac Jessa,Magandang araw po saan po pwede lumapit kung m...
26216,202207211607,2022-07-21,Muslimah Balik Islam,From U.A.E pano po mag aply s owwa scholarship
26217,202207211608,2022-07-21,Lina Gabis,Sir tanong lang nagrenew kasi kami ng mga kasa...


In [6]:
# drop null rows
all_df = all_df[~all_df['text'].isna()].reset_index(drop=True)

### Clean comments

In [7]:
cleaner = CommentCleaner()
len(cleaner.get_padwords())

31

In [8]:
all_df['clean_text'] = all_df['text'].apply(lambda x: cleaner.clean_comment(x))
all_df['clean_text_word_count'] = all_df['clean_text'].apply(lambda x: len(x.split(' ')))

In [9]:
all_df['text_word_count'] = all_df['text'].apply(lambda x: len(x.split(' ')))

In [10]:
# remove comments fewer than 6 words
all_df = all_df[all_df['text_word_count'] >= 6].reset_index(drop=True)
len(all_df)

19032

In [11]:
# remove comments empty text after cleaning
all_df = all_df[all_df['clean_text_word_count'] >= 2].reset_index(drop=True)
len(all_df)

17536

In [31]:
all_df[all_df['text'].str.contains('xa')]['text'].values

array(['Hirap mag comment!! Kasi view and followers lng Naman habol nila dyan kahit lumapit kpa xa pollo huo ika nga may kunting tulong pero double Ang balik xa kanila , Sana maging matalino na Tayo mga OFW',
       'Sir/maam pede paba maka avail nang scholarship yong anak ko college na xa 21 yrs old na',
       'Edsp Yan Yung 601 dollar pataas pag nakapasa ka SA exam 60 Tau yearly',
       'SIr tanong kulang kilan mag start sa CARAGA EDSP entrance exam, till now wala pa din,',
       'Odsp 600 pababa sahod ni Ofw .. walang exam 20k yearly',
       'May exam po ba kasi last year wala namang sinabing exam',
       'Urgent po hingi po ako ng 2long d npo xa pinapakain at nkakapag pahinga ng maayos gus2 nya npo 2makas s hirap at gu2m',
       'Sir ako po nakakuha po ng balik pinas hanap buhay at waiting for filing po anak ko ng scholarship 93 po average nya po last grade 12 po xa',
       'Sir saan po b pwde mg exam pra sa scholarship ng owwa',
       '0945417397..sana po matawagan at matu

In [12]:
all_df['clean_text_word_count'].describe()

count    17536.000000
mean         9.062044
std          8.038674
min          2.000000
25%          4.000000
50%          7.000000
75%         11.000000
max        207.000000
Name: clean_text_word_count, dtype: float64

### Guess gender of comment sender

In [13]:
gg = GenderGuesser()

In [17]:
#func_result = [(x,gg.guess_gender(x)) for x in all_df['sender_name'].values[0:100]]

In [19]:
all_df['gender'] = all_df['sender_name'].apply(lambda x: gg.guess_gender(x))
all_df['gender'].value_counts()

feminine            8573
unknown             5402
masculine           2523
androgynous          456
mostly_feminine      317
mostly_masculine     265
Name: gender, dtype: int64

In [20]:
all_df

,comment_id,date_aired,sender_name,text,clean_text,clean_text_word_count,text_word_count,gender
0,202207070061,2022-07-07,John Godfrey,Palala lamang po sa mga ofws sa ksa wag mameke...,palala ofws ksa wag mameke dokyumento nahuli n...,8,17,masculine
1,202207070065,2022-07-07,Agripina Esmajer,Hi Sir Arnel good evening Po!watching from San...,joaquin iloilo yrs dole akap financial assista...,8,22,feminine
2,202207070066,2022-07-07,Merybeth Sumagpao,Hello sir and ma'am Sana matulungan nyo nman a...,matulungan 14year abroad pandemic hindi balik ...,12,32,feminine
3,202207070067,2022-07-07,Airam Ocab Aziter,"Sir arnel and all the staff, Good evening po! ...",staff ofw kuwait years member owwa lockdown 20...,13,37,feminine
4,202207070068,2022-07-07,Karen May Mendiola,Paano Kong Hindi Naman distress gusto na lng t...,hindi distress gusto 22yrs lebanon active owwa...,12,35,feminine
...,...,...,...,...,...,...,...,...
17531,202207211605,2022-07-21,Sarupac Jessa,Magandang araw po saan po pwede lumapit kung m...,araw lumapit repatriation reuest agency oman h...,19,57,unknown
17532,202207211606,2022-07-21,Sarupac Jessa,Magandang araw po saan po pwede lumapit kung m...,araw lumapit repatriation reuest agency oman h...,21,51,unknown
17533,202207211607,2022-07-21,Muslimah Balik Islam,From U.A.E pano po mag aply s owwa scholarship,aply owwa scholarship,3,9,feminine
17534,202207211608,2022-07-21,Lina Gabis,Sir tanong lang nagrenew kasi kami ng mga kasa...,tanong nagrenew kasama polo owwa araw bago pap...,12,27,feminine


In [ ]:
all_df['sender_name'] = all_df['sender_name'].apply(lambda x: cleaner.censor_name(x))
all_df['text'] = all_df['text'].apply(lambda x: cleaner.censor_text(x))
all_df.to_csv('../output/all_comments.csv', index=False)